In [1]:
import sys
import cobra
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
import cplex
import cobrakbase
#Put the path to ModelSEEDpy on your machine here
sys.path.append("/Users/chenry/code/ModelSEEDpy")
#import modelseedpy.fbapkg
from modelseedpy import MSPackageManager, FBAHelper, MSBuilder


cobrakbase 0.2.7


In [2]:
kbase_api = cobrakbase.KBaseAPI()
#List available models
modellist = kbase_api.ws_client.list_objects({
    "ids":[93541],
    "type":"KBaseFBA.FBAModel"
})
#Iterate over the models and run FBA
LBMedia = model = kbase_api.get_from_ws("ArgonneLBMedia","KBaseMedia")
MMMedia = model = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")
template = kbase_api.get_from_ws("GramNegModelTemplateV3","NewKBaseModelTemplates")
results = {}
count = 0
for item in modellist:
    model = kbase_api.get_from_ws(item[1],item[6])
    results[item[1]] = {"complete":{"genes":None,"reactions":None},"LB":{"genes":None,"reactions":None},"MM":{"genes":None,"reactions":None}}
    FBAHelper.set_objective_from_target_reaction(model, "bio1")
    pkgmgr = MSPackageManager.get_pkg_mgr(model)
    pkgmgr.getpkg("KBaseMediaPkg").build_package(None)
    solution=model.optimize()
    if solution.objective_value == 0:
        print("Gapfilling "+item[1]+" in complete media")
        model = MSBuilder.gapfill_model(model,"bio1",template,None)
    print("Computing essentiality for "+item[1]+" in complete media")
    results[item[1]]["complete"]["genes"] = single_gene_deletion(model)
    results[item[1]]["complete"]["genes"] = single_reaction_deletion(model)
    pkgmgr.getpkg("KBaseMediaPkg").build_package(LBMedia)
    solution=model.optimize()
    if solution.objective_value == 0:
        print("Gapfilling "+item[1]+" in LB media")
        model = MSBuilder.gapfill_model(model,"bio1",template,LBMedia)
        pkgmgr = MSPackageManager.get_pkg_mgr(model)
        pkgmgr.getpkg("KBaseMediaPkg").build_package(LBMedia)
    print("Computing essentiality for "+item[1]+" in LB media")
    results[item[1]]["LB"]["genes"] = single_gene_deletion(model)
    results[item[1]]["LB"]["reactions"] = single_reaction_deletion(model)
    pkgmgr.getpkg("KBaseMediaPkg").build_package(MMMedia)
    solution=model.optimize()
    if solution.objective_value == 0:
        print("Gapfilling "+item[1]+" in MM media")
        model = MSBuilder.gapfill_model(model,"bio1",template,MMMedia)
        pkgmgr = MSPackageManager.get_pkg_mgr(model)
        pkgmgr.getpkg("KBaseMediaPkg").build_package(MMMedia)
    print("Computing essentiality for "+item[1]+" in MM media")
    results[item[1]]["MM"]["genes"] = single_gene_deletion(model)
    results[item[1]]["MM"]["reactions"] = single_reaction_deletion(model)
    count += 1
    if count > 2:
        break


Computing essentiality for 562.61280.fbamodel in complete media
Gapfilling 562.61280.fbamodel in LB media
Computing essentiality for 562.61280.fbamodel in LB media
Gapfilling 562.61280.fbamodel in MM media
Computing essentiality for 562.61280.fbamodel in MM media
Computing essentiality for 562.55367.fbamodel in complete media
Gapfilling 562.55367.fbamodel in LB media
Computing essentiality for 562.55367.fbamodel in LB media
Gapfilling 562.55367.fbamodel in MM media
Computing essentiality for 562.55367.fbamodel in MM media
Computing essentiality for 562.55368.fbamodel in complete media
Gapfilling 562.55368.fbamodel in LB media
Computing essentiality for 562.55368.fbamodel in LB media
Gapfilling 562.55368.fbamodel in MM media
Computing essentiality for 562.55368.fbamodel in MM media
